First, the required libraries are imported and the files are read and merged so that the missions also have the characteristics of the planets they refer to. The mission number for each planet is also calculated.

In [8]:
# Import libraries
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import math

In [9]:
# Access planet data
planets = pd.read_csv('/project/Datasets/planets.csv')

planets

,name,diameter,color,distance_from_sun,distance_from_earth,regularized_radius,regularized_distance_from_sun,regularized_distance_from_earth
0,Sun,432169,#f6d23f,0.000000e+00,-92956050,20,0.0,-170
1,Mercury,1516,#c9c3c3,3.598312e+07,-56972925,2,60.0,-110
2,Venus,3760,#a37848,6.723825e+07,-25717799,4,120.0,-50
3,Earth,3959,#447ebf,9.295605e+07,0,4,170.0,0
4,Moon,2159,#9a9796,NaN,247260,2,NaN,20
5,Mars,2106,#b45034,1.416377e+08,48681675,2,230.0,60
6,Jupiter,43441,#b89a7b,4.836386e+08,390682514,10,330.0,160
7,Saturn,36184,#d2ba81,8.864894e+08,793533365,10,430.0,260
8,Uranus,15759,#c2e6e9,1.783744e+09,1690788250,6,550.0,380
9,Neptune,15299,#4575fb,2.795174e+09,2702217910,6,680.0,510


In [10]:
# Access missions data
missions = pd.read_csv('/project/Datasets/missions.csv')

# Merge missions with planets
missions = missions.merge(planets[['name', 'regularized_radius', 'regularized_distance_from_sun', 'regularized_distance_from_earth']], left_on = 'planet', right_on = 'name').drop(columns = ['name'])

# Count cumulative number of missions per planet
missions['planet_mission_number'] = missions.groupby('planet')['planet'].cumcount().add(1)

missions.head()

,company,location,country,date,year,rocket,rocket_status,mission,outcome,crewed,destination,planet,main_astronaut,details,regularized_radius,regularized_distance_from_sun,regularized_distance_from_earth,planet_mission_number
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",Kazakhstan,"Fri Oct 04, 1957 19:28 UTC",1957,Sputnik 8K71PS,Retired,Sputnik-1,Success,No,Earth orbit,Earth,NaN,Humankind's first artificial satellite,4,170.0,0,1
1,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",USA,"Sat Feb 01, 1958 03:48 UTC",1958,Juno I,Retired,Explorer 1,Success,No,Earth orbit,Earth,NaN,US launches first satellite,4,170.0,0,2
2,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",Kazakhstan,"Wed Apr 12, 1961 06:07 UTC",1961,Vostok,Retired,Vostok 1,Success,Yes,Earth orbit,Earth,Yuri Gagarin,First human to fly into space and return to Ea...,4,170.0,0,3
3,NASA,"LC-14, Cape Canaveral AFS, Florida, USA",USA,"Tue Feb 20, 1962 14:47 UTC",1962,Atlas-D Mercury,Retired,Friendship 7 (MA-6),Success,Yes,Earth orbit,Earth,John Glenn,First American to orbit Earth,4,170.0,0,4
4,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",Kazakhstan,"Sun Jun 16, 1963 09:29 UTC",1963,Vostok,Retired,Vostok 6,Success,Yes,Earth orbit,Earth,Valentina Tereshkova,First woman in space,4,170.0,0,5


In [11]:
# Count total number of missions per planet
planet_missions = missions.groupby('planet')[['planet_mission_number']].max().reset_index().rename(columns = {'planet':'name', 'planet_mission_number':'number_of_missions'})
planets = planets.merge(planet_missions, how = 'left', on = 'name')
planets = planets.fillna(0)
planets.number_of_missions = planets.number_of_missions.astype(int)


Then, functions for plotting 3d objects defined. The mathematical equations for these were gathered online (Dobilas, 2020). Sphere is a function that plots the surface of a sphere. θ and φ are defined as matrices with 100 numbers between [0, 2π] and [0, π] respectivelly. Ones is a matrix with length 100 containing number 1 everywhere. Then the coordinates of the points of the surface of the sphere are calculated by the following formulas:

$$ x = x_{sphere} + radius_{sphere} * matrix\_multiplication(cos(θ), sin(φ)) $$

$$ y = y_{sphere} + radius_{sphere} * matrix\_multiplication(sin(θ), sin(φ)) $$

$$ z = z_{sphere} + radius_{sphere} * matrix\_multiplication(ones, sin(φ)) $$

The surface connecting the above coordinates is constructed as a plolty Surface object, with its color fading to give a 3d perspective to the object. Its name set to appear when hovering on it.

In [12]:
def sphere(x_0, y_0, z_0, radius, color, name = ''): 
    '''
    Plots the surface of a sphere

    x_0: X coordinate
    y_0: Y coordinate
    z_0: Z coordinate
    radius: Radius of sphere
    color: Color of sphere
    name: Name to be printed on hover
    '''
    # Calculate theta and phi
    theta = np.linspace(start = 0,
                        stop = 2 * np.pi,
                        num = 100)
    phi = np.linspace(start = 0,
                      stop = np.pi,
                      num = 100)
    ones = np.ones(100)
    
    # Set coordinates of points on the sphere
    x_crd = x_0 + radius * np.outer(np.cos(theta), np.sin(phi))
    y_crd = y_0 + radius * np.outer(np.sin(theta), np.sin(phi))
    z_crd = z_0 + radius * np.outer(ones,np.cos(phi))
    
    # Return surface trace
    trace = go.Surface(name = name,
                       x = x_crd,
                       y = y_crd,
                       z = z_crd,
                       colorscale = [[0, color], [1, color]],
                       hoverinfo = 'name',
                       showscale = False)
    
    return trace

Similarly, the function orbit plots a part of the orbit of a shpere. First, every given angle is transformed into a radian. Then, the coordinates of each point of the orbit are calculated using the formulas below:

$$ x = x_{sphere} + radius_{sphere} * cos(radian)$$

$$ y = y_{sphere} + radius_{sphere} * sin(radian)$$

Coordinate z is set to the same as the planet's, as the orbit is in the same height. The above points are constructed as a plolty Scatter3d object and its name set not to appear when hovering.

In [13]:
def orbit(x_0, y_0, z_0, radius, color, from_angle = 0, to_angle = 361): 
    '''
    Plots a part of the orbit of a shpere
    
    x_0: X coordinate
    y_0: Y coordinate
    z_0: Z coordinate
    radius: Radius of orbit ricle
    color: Color of orbit circle
    from_angle: Start angle of orbit
    to_angle: End angle of orbit
    '''
    # Define empty lists for coordinates
    x_crd = []
    y_crd = []
    z_crd = []

    # Set coordinates of orbit
    for i in range(from_angle, to_angle):
        radian = math.radians(i)
        
        x_crd += [x_0 + radius * (round(np.cos(radian), 5))]
        y_crd += [y_0 + radius * (round(np.sin(radian), 5))]
        z_crd += [z_0]

    # Return orbit trace
    trace = go.Scatter3d(name = '',
                         x = x_crd,
                         y = y_crd,
                         z = z_crd,
                         marker = dict(size = 0.1),
                         line = dict(color = color, width = 2),
                         hoverinfo = 'name')
    
    return trace

Function annotation is used to plot the name of planets and missions at specific coordinates.

In [14]:
def annotation(x_0, y_0, z_0, name):
    '''
    Returns dictionary with details about the annotation of an object
    
    x_0: X coordinate
    y_0: Y coordinate
    z_0: Z coordinate
    text: The name of the object
    '''
    annot = dict(showarrow = False,
                 x = x_0,
                 y = y_0,
                 z = z_0,
                 text = name,
                 xanchor = 'center',
                 font = dict(color = 'white', size = 10))
    
    return annot

The planets are visualised based on their coordinates, size and color, with Earth set at (0, 0, 0) in order to be in the middle of the visualisation. Their names are constructed as objects on top of them. As regards each planet's orbit, it is centered around the Sun and ranges from 0 to 90 degrees with it's color fading as the distance to the planet increases. This is achieved by constructing 9 traces with different colors for each orbit, one for every 10 degrees. The total number of missions of each planet is set to appear when hovering on it. Some exceptions worth mentioning are Saturn and Moon. Saturn's rings rings are centered on the planet's coordinates and have a consistent color along degrees 0 to 360. Moon, on the other hand, is not alligned with the other planets and orbits Earth instead.

In [15]:
# Define empty lists for objects and their annotations
traces = []
annotations = []

In [16]:
# Define ranges for planet orbits and their corresponding colors
ranges_from = list(range(0, 81, 10))
ranges_to = list(range(11, 91, 10))
colors = ['#808080', '#707070', '#606060', '#505050', '#404040', '#303030', '#202020', '#101010', '#050505']

# Planets traces preparation
for i in range(len(planets)):
    # Define planet coordinates, with only the moon being not alligned
    x = planets.regularized_distance_from_earth[i]
    if planets.name[i] == 'Moon':
        y = planets.regularized_distance_from_earth[i]
    else:
        y = 0
    z = 0
    
    # Define other planet characteristics
    radius = planets.regularized_radius[i]
    color = planets.color[i]
    name = planets.name[i]
    hover = 'Number of missions: ' + str(planets.number_of_missions[i])
    
    # Construct planets and their annotations
    planet_trace = sphere(x, y, z, radius, color, hover)
    planet_annotation = annotation(x, y, z + radius + 3, name)
    
    # Constuct Saturn rings
    if name == 'Saturn':
        for j in range(3, 14, 2):
            ring_trace = orbit(x, y, z, radius + j, color)
            traces.append(ring_trace)
    
    # Construct planet orbits. Range 0 - 10 degrees has color '#808080', 10 - 20 has '#707070' etc
    for range_from, range_to, color in zip(ranges_from, ranges_to, colors):
        # The moon orbits Earth
        if name == 'Moon':
            orbit_trace = orbit(0, 0, 0, x + 9, color, range_from + 45, range_to + 45)
        # Other planets orbit Sun
        else:
            sun_x = int(planets[planets.name == 'Sun'].regularized_distance_from_earth)
            orbit_trace = orbit(sun_x, 0, 0, planets.regularized_distance_from_sun[i], color, range_from, range_to)
        traces.append(orbit_trace)
    
    # Save constructed objects
    traces.append(planet_trace)
    annotations.append(planet_annotation)

As regards the missions, they are visualised as shperes on top of the planet they refer to, starting from the oldest to the most recent. Similarly to planets, their names constitute objects and their country, year and description are set to appear on hover.

In [17]:
# Missions traces preparation
for i in range(len(missions)):
    # Define mission coordinates to be on top of their corresponding planets
    x = missions.regularized_distance_from_earth[i]
    if missions.planet[i] == 'Moon':
        y = missions.regularized_distance_from_earth[i]
    else:
        y = 0
    z = missions.regularized_radius[i] + 5 + 11 * missions.planet_mission_number[i]
    
    # Define other planet characteristics
    radius = 0.5
    color = 'dimgray'
    name = str(missions.mission[i])
    hover = 'Country: ' + str(missions.country[i]) + '<br>Year: ' + str(missions.year[i]) + '<br>' + str(missions.details[i])
    
    # Construct missions and their annotations
    mission_trace = sphere(x, y, z, radius, color, hover)
    mission_annotation = annotation(x, y, z + radius + 3, name)
    
    # Save constructed objects
    traces.append(mission_trace)
    annotations.append(mission_annotation)

The appearance of the graph is set as follows: The ranges of the axes allow enough space to host the aforementioned objects and all the unecessary information, like then legend and axes are hidden from the background. The final graph is exported as an html file.

In [18]:
# Customise axes
axis_customisation = dict(range = [-681, 681],
                          # Hide everything in the background
                          color = 'black',
                          backgroundcolor = 'black',
                          gridcolor = 'black',
                          titlefont_color = 'black',
                          zeroline = False)

# Customise layout
layout = go.Layout(title = 'Most important space missions',
                   scene = dict(xaxis = axis_customisation,
                                yaxis = axis_customisation,
                                zaxis = axis_customisation,
                                annotations = annotations),
                   # Display whole text when hovering
                   hoverlabel = dict(namelength = -1),
                   # Hide everything in the background
                   paper_bgcolor = 'black',
                   coloraxis = None,
                   showlegend = False)

# Prepare plot
fig = go.Figure(data = traces,
                layout = layout)

# Export to HTML
fig.write_html("Missions.html")

In [22]:
import chart_studio.tools as tls
tls.get_embed('https://raw.githubusercontent.com/KonstantinosKoumentakis/space_missions/main/Missions.html')

PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, 'https://raw.githubusercontent.com/KonstantinosKoumentakis/space_missions/main/Missions.html', we expected it to start with 'https://plotly.com'.
Run help on this function for more information.

# References

Dobilas, S., 2020. How to Create a 3D Model of the Solar System with Plotly in Python. [online] Medium. Available at: <https://python.plainenglish.io/how-to-create-a-3d-model-of-the-solar-system-with-plotly-in-python-2a74e672b771> [Accessed 20 March 2022].